In [140]:
import pandas as pd
import json
import requests
from python_secrets import *

In [142]:
df_main = pd.read_csv('incident2.csv')

In [143]:
tx_hash_list = [x for x in df_main[' user_tx'].to_list() if pd.notnull(x)]

# Infuria

In [144]:
url = f"https://mainnet.infura.io/v3/{infura_api_key}"

payload = json.dumps({
  "jsonrpc": "2.0",
  "method": "eth_getTransactionByHash",
  "params": [tx_hash_list[0]],
  "id": 1
})
headers = {
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)
dct = response.json()['result']
dct = {k: None if not v else v for k, v in dct.items()} # making sure none of the values are empty
df_infuria = pd.DataFrame(dct, index=[0])

for tx_hash in tx_hash_list[1:]:
  payload = json.dumps({
    "jsonrpc": "2.0",
    "method": "eth_getTransactionByHash",
    "params": [tx_hash],
    "id": 1
  })
  headers = {
    'Content-Type': 'application/json'
  }

  response = requests.request("POST", url, headers=headers, data=payload)

  if response.ok:
    dct = response.json()['result']
    dct = {k: None if not v else v for k, v in dct.items()} # making sure none of the values are empty
    df_temp = pd.DataFrame(dct, index=[0])
    df_infuria = pd.concat([df_infuria, df_temp])
    
  else: 
    print(f"error code {response.status_code} for transaction {tx_hash}")


In [145]:
#merge infuria to main
df_main = df_main.merge(df_infuria, left_on = ' user_tx', right_on = 'hash', how ='outer')

In [146]:
df_main.columns

Index(['block_number', ' user_tx', ' fees', 'accessList', 'blockHash',
       'blockNumber', 'chainId', 'from', 'gas', 'gasPrice', 'hash', 'input',
       'maxFeePerGas', 'maxPriorityFeePerGas', 'nonce', 'r', 's', 'to',
       'transactionIndex', 'type', 'v', 'value', 'yParity'],
      dtype='object')

# Etherscan 

In [147]:
block_number_list = list(set([x for x in df_main['block_number'].to_list() if pd.notnull(x)]))

In [150]:
url_eth = f"https://api.etherscan.io/api?module=block&action=getblockreward&blockno={block_number_list[0]}&apikey={eth_scan_api_key}"

response_eth = requests.request("POST", url_eth)

dct_eth = response_eth.json()['result']
dct_eth = {k: None if not v else v for k, v in dct_eth.items()} # making sure none of the values are empty
df_eth = pd.DataFrame(dct_eth, index=[0])

for block in block_number_list[1:]:
  url_temp = f"https://api.etherscan.io/api?module=block&action=getblockreward&blockno={block}&apikey={eth_scan_api_key}"
  response_temp = requests.request("POST", url_temp)

  if response_temp.ok:
    dct_temp = response_temp.json()['result']
    dct_temp = {k: None if not v else v for k, v in dct_temp.items()} # making sure none of the values are empty
    df_temp = pd.DataFrame(dct_temp, index=[0])
    df_eth = pd.concat([df_eth, df_temp])
    
  else: 
    print(f"error code {response_temp.status_code} for block {block}")

In [157]:
df_eth['blockNumber'] = df_eth['blockNumber'].astype(int)

In [158]:
#merge eth to main
df_main = df_main.merge(df_eth, left_on = 'block_number', right_on = 'blockNumber', how ='outer')

# Tenderly

In [163]:
len(tx_hash_list)

37

In [164]:
df_main.columns

Index(['block_number', ' user_tx', ' fees', 'accessList', 'blockHash',
       'blockNumber_x', 'chainId', 'from', 'gas', 'gasPrice', 'hash', 'input',
       'maxFeePerGas', 'maxPriorityFeePerGas', 'nonce', 'r', 's', 'to',
       'transactionIndex', 'type', 'v', 'value', 'yParity', 'blockNumber_y',
       'timeStamp', 'blockMiner', 'blockReward', 'uncles',
       'uncleInclusionReward'],
      dtype='object')

In [278]:
columns = ['tx_hash', 'index', 'type', 'raw_amount', 'dollar_value', 'token_contract_address', 'token_name', 'token_dollar_value']
df_results = pd.DataFrame(columns = columns)

for index, row in df_main.iterrows():
    tx_index_list = [0]
    tx_index_list.append(int(row['transactionIndex'], 0))
    print(tx_index_list)
    for tx_index in tx_index_list:
        print(tx_index)
        json_data = {
        'network_id': int(row['chainId'], 0),
        'from': row['from'],
        'to': row['to'],
        'input': row['input'],
        'block_number': row['block_number'],
        'transaction_index': tx_index,
        'simulation_type': 'quick'
    }
        response = requests.post(
        'https://api.tenderly.co/api/v1/account/aurelie/project/cowswap/simulate',
        headers=headers,
        json=json_data,
        )
        print(response.json())

        try:
            for data in response.json()['transaction']['transaction_info']['asset_changes']:
                print(data)
                tx_type = data['type']
                tx_raw_amount = data['raw_amount']
                tx_dollar_value = data['dollar_value']

                try:
                    contract_address = data['token_info']['contract_address']
                except:
                    contract_address = 'None'

                try:
                    token_name = data['token_info']['name']
                except:
                    token_name = 'None'
                try:
                    token_dollar_value = data['token_info']['dollar_value']
                except:
                    token_dollar_value = 'None'

                new_row = {
                    'tx_hash' : row['hash'],
                    'index' : tx_index, 
                    'type': tx_type, 
                    'raw_amount': tx_raw_amount, 
                    'dollar_value' : tx_dollar_value, 
                    'token_contract_address': contract_address, 
                    'token_name': token_name, 
                    'token_dollar_value': token_dollar_value
                    }
                df_results = pd.concat([df_results, pd.DataFrame([new_row])], ignore_index=True)
        except:
            df_main[df_main['input'] == '']['hash']
            


[0, 2]
0
{'transaction': {'hash': '0xd9488fe37e24c1b3f14937e3029036e6aa480d1e1621d81a96ef0779c492f64b', 'block_hash': '', 'block_number': 19412019, 'from': '0x36a2ffb33c1b427c46c3d30adac3ca4e8ed36179', 'gas': 9223372036854775807, 'gas_price': 0, 'gas_fee_cap': 0, 'gas_tip_cap': 0, 'cumulative_gas_used': 0, 'gas_used': 142396, 'effective_gas_price': 0, 'input': '0x4a25d94a000000000000000000000000000000000000000000000000058d15e17628000000000000000000000000000000000000000000000000002f461b77a211c3b3ee00000000000000000000000000000000000000000000000000000000000000a000000000000000000000000036a2ffb33c1b427c46c3d30adac3ca4e8ed361790000000000000000000000000000000000000000000000000000000065ef077b0000000000000000000000000000000000000000000000000000000000000002000000000000000000000000b0699d63aef20df3f1cffa9ca2bb8670416271d2000000000000000000000000c02aaa39b223fe8d0a0e5c4f27ead9083c756cc2', 'nonce': 1799, 'to': '0x7a250d5630b4cf539739df2c5dacb4c659f2488d', 'index': 0, 'value': '0x', 'access_list': No

TypeError: 'NoneType' object is not iterable

In [280]:
df_main[df_main['input'] == '']['hash']

1    0x354e8386267ca643793de913739df3f9680895776f3a...
Name: hash, dtype: object

In [279]:
response.json()['transaction']['transaction_info']#['asset_changes']

{'contract_id': 'eth:1:0x89b69f2d1adffa9a253d40840b6baa7fc903d697',
 'block_number': 19412030,
 'transaction_id': '0xa247f76862dac29014f9979c749646c1758a613b5c7fad6158814f58e9cb91b3',
 'contract_address': '0x89b69f2d1adffa9a253d40840b6baa7fc903d697',
 'method': None,
 'parameters': None,
 'intrinsic_gas': 27656,
 'refund_gas': 0,
 'call_trace': {'hash': '0xa247f76862dac29014f9979c749646c1758a613b5c7fad6158814f58e9cb91b3',
  'contract_name': '',
  'function_pc': 0,
  'function_op': 'CALL',
  'absolute_position': 0,
  'caller_pc': 0,
  'caller_op': 'CALL',
  'call_type': 'CALL',
  'address': '0x4ffb89a61a6db0586aff308efcfce39207aed2b2',
  'from': '0x4ffb89a61a6db0586aff308efcfce39207aed2b2',
  'from_balance': '76427735822750178',
  'to': '0x3fc91a3afd70395cd496c647d5a6cc9d4b2b7fad',
  'to_balance': '0',
  'value': '0',
  'block_timestamp': '0001-01-01T00:00:00Z',
  'gas': 9223372036854748151,
  'gas_used': 129479,
  'intrinsic_gas': 27656,
  'storage_address': '0x4ffb89a61a6db0586aff308e